In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai chromadb bs4

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import getpass
import os
os.environ['USER_AGENT']='ProfessorAgent'

import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

In [11]:
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Vald
from langchain_text_splitters import CharacterTextSplitter

raw_documents = TextLoader("../data/data.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=3000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)

In [12]:
docs = text_splitter.split_documents(documents)

In [13]:
len(docs[0].page_content)

324993

In [14]:
print(docs[0].page_content[5000:8000])

tion virtualization because I will come back to this once I explained Holograms and volumetric media delivery then I will also Explain how virtual Network functions can be of interest so if we consider a home environments then there are multiple devices and in your home and typically there is a home Gateway. And on the home K3, we have different functionality. There is the HTTP server. There is an entity router firewall also quality of experience monitoring software for instance. if we have TV streams or video streams that we display on screens inside your home then we also monitor the quality of experience inside the home Gateway. And then typical Auto device is a setup box. so stb. Which contains? for a digital TV storage so that you can record movies and TV programs. Middleware that allows you to interact through remote control. And to start recording stop recording play a certain TV program. and so on for unless we need to middleware on the setup box. And also we have a streaming c

In [18]:
embeddings = OpenAIEmbeddings()
db = DocArrayInMemorySearch.from_documents(docs, embeddings)

In [19]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

In [20]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

In [21]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI


llm = ChatOpenAI(temperature=0.0, model="gpt-3.5-turbo", max_tokens=2048)
db = DocArrayInMemorySearch.from_documents(
    all_splits,
    embeddings
)
retriever = db.as_retriever()
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    verbose=True
)

In [22]:
from langchain.agents import tool

@tool
def find_seminar_quote(text: str) -> str:
    """ Receives a question from the user about a seminar and returns a description of it from the transcript.
    """

    query =  f"""Please gather more information about '''{text}''' and summarize the output"""
    response = qa_stuff.run(query)
    
    return response


In [23]:
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain_community.tools.tavily_search import TavilySearchResults

tools=[find_seminar_quote]
tools = tools # +  [TavilySearchResults(max_results=1)]# + load_tools(["wikipedia"], llm=llm)
agent= initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

/home/pedro/Desktop/Projects/seminars-agent/env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [35]:
agent("Based on what you know what is the most efficient techinique to prompt an llm?")

/home/pedro/Desktop/Projects/seminars-agent/env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Thought: I should use the `find_seminar_quote` tool to gather information on the most efficient technique to prompt an LLM.

Action:
```
{
  "action": "find_seminar_quote",
  "action_input": "most efficient technique to prompt an LLM"
}
```

> Entering new RetrievalQA chain...


/home/pedro/Desktop/Projects/seminars-agent/env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(



> Finished chain.

Observation: I don't have enough information to provide a specific answer to your question about the most efficient technique to prompt an LLM. The context provided discusses the importance of formatting or prompting engineering in optimizing code and utilizing autocomplete systems effectively. It mentions the use of demonstrations, chaining thoughts, and editing to enhance the efficiency of prompting. Additionally, it suggests avoiding training your own base models and utilizing cloud APIs for instruction tuning. If you are interested in exploring this topic further, you may want to look into the GitHub repository mentioned in the context for more detailed information.
Thought:Could not parse LLM output: Final Answer
Observation: Invalid or incomplete response
Thought:I will try using the `find_seminar_quote` tool again to gather more information on the most efficient technique to prompt an LLM.

Action:
```
{
  "action": "find_seminar_quote",
  "action_input": "ef

{'input': 'Based on what you know what is the most efficient techinique to prompt an llm?',
 'output': "The most efficient technique to prompt a language model involves using few-shot prompting and instruction tuning to enhance the model's performance."}

In [25]:
agent("How holograms will be sent in our networks?")



> Entering new AgentExecutor chain...
Thought: This question seems to be asking about the transmission of holograms in networks. I should use the find_seminar_quote tool to see if there is any relevant information in the seminar transcript.

Action:
```
{
  "action": "find_seminar_quote",
  "action_input": "holograms sent in networks"
}
```

> Entering new RetrievalQA chain...

> Finished chain.

Observation: Based on the provided context, holograms being sent over networks require high bandwidth, on the order of gigabits per second. This is due to the complex data involved in holographic communications. Two important technologies for transmitting holograms are Point Clouds and Light Fields. Point Clouds involve representing objects by considering all possible points in three-dimensional space, while Light Fields technology is expected to be important in the future for modeling environments as well as objects. The bandwidth requirements for transmitting video over networks vary depen

{'input': 'How holograms will be sent in our networks?',
 'output': "The transmission of holograms in networks requires high bandwidth, on the order of gigabits per second, and compression techniques are essential for efficient data transport. Holograms in networks involve volumetric media like holographic images or videos and have applications in various sectors. The Spirit project in Europe explores holographic communications' potential in areas such as healthcare, retail, education, entertainment, and more."}